In [12]:
import random
import subprocess
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import defaultdict

sns.set(style="whitegrid")

![Image](images/original.png) 

![Image](images/bert-like-before.png) 

![Image](images/bert-like-after.png) 

In [13]:
#only column sparsity
runs = [["0.0", "0.1", "SparseDense", "0"], ["0.0", "0.3", "SparseDense", "0"], ["0.0", "0.5", "SparseDense", "0"], ["0.0", "0.7", "SparseDense", "0"], ["0.0", "0.9", "SparseDense", "0"], ["0.0", "0.1", "SparseDense", "1"], ["0.0", "0.3", "SparseDense", "1"], ["0.0", "0.5", "SparseDense", "1"], ["0.0", "0.7", "SparseDense", "1"], ["0.0", "0.9", "SparseDense", "1"], ["0.0", "0.1", "DD", "0"], ["0.0", "0.3", "DD", "0"], ["0.0", "0.5", "DD", "0"], ["0.0", "0.7", "DD", "0"], ["0.0", "0.9", "DD", "0"]]

#only row sparsity
#runs = [["0.1", "0.0", "SparseDense", "0"], ["0.3", "0.0", "SparseDense", "0"], ["0.5", "0.0", "SparseDense", "0"], ["0.7", "0.0", "SparseDense", "0"], ["0.9", "0.0", "SparseDense", "0"], ["0.1", "0.0", "SparseDense", "1"], ["0.3", "0.0", "SparseDense", "1"], ["0.5", "0.0", "SparseDense", "1"], ["0.7", "0.0", "SparseDense", "1"], ["0.9", "0.0", "SparseDense", "1"], ["0.1", "0.0", "DD", "0"], ["0.3", "0.0", "DD", "0"], ["0.5", "0.0", "DD", "0"], ["0.7", "0.0", "DD", "0"], ["0.9", "0.0", "DD", "0"]]

#row and column sparsity
#runs = [["0.1", "0.1", "SparseDense", "0"], ["0.3", "0.3", "SparseDense", "0"], ["0.5", "0.5", "SparseDense", "0"], ["0.7", "0.7", "SparseDense", "0"], ["0.9", "0.9", "SparseDense", "0"], ["0.1", "0.1", "SparseDense", "1"], ["0.3", "0.3", "SparseDense", "1"], ["0.5", "0.5", "SparseDense", "1"], ["0.7", "0.7", "SparseDense", "1"], ["0.9", "0.9", "SparseDense", "1"], ["0.1", "0.1", "DD", "0"], ["0.3", "0.3", "DD", "0"], ["0.5", "0.5", "DD", "0"], ["0.7", "0.7", "DD", "0"], ["0.9", "0.9", "DD", "0"]]

#all
#runs = [["0.0", "0.1", "SparseDense", "0"], ["0.0", "0.3", "SparseDense", "0"], ["0.0", "0.5", "SparseDense", "0"], ["0.0", "0.7", "SparseDense", "0"], ["0.0", "0.9", "SparseDense", "0"], ["0.0", "0.1", "SparseDense", "1"], ["0.0", "0.3", "SparseDense", "1"], ["0.0", "0.5", "SparseDense", "1"], ["0.0", "0.7", "SparseDense", "1"], ["0.0", "0.9", "SparseDense", "1"], ["0.0", "0.1", "DD", "0"], ["0.0", "0.3", "DD", "0"], ["0.0", "0.5", "DD", "0"], ["0.0", "0.7", "DD", "0"], ["0.0", "0.9", "DD", "0"], ["0.1", "0.0", "SparseDense", "0"], ["0.3", "0.0", "SparseDense", "0"], ["0.5", "0.0", "SparseDense", "0"], ["0.7", "0.0", "SparseDense", "0"], ["0.9", "0.0", "SparseDense", "0"], ["0.1", "0.0", "SparseDense", "1"], ["0.3", "0.0", "SparseDense", "1"], ["0.5", "0.0", "SparseDense", "1"], ["0.7", "0.0", "SparseDense", "1"], ["0.9", "0.0", "SparseDense", "1"], ["0.1", "0.0", "DD", "0"], ["0.3", "0.0", "DD", "0"], ["0.5", "0.0", "DD", "0"], ["0.7", "0.0", "DD", "0"], ["0.9", "0.0", "DD", "0"], ["0.1", "0.1", "SparseDense", "0"], ["0.3", "0.3", "SparseDense", "0"], ["0.5", "0.5", "SparseDense", "0"], ["0.7", "0.7", "SparseDense", "0"], ["0.9", "0.9", "SparseDense", "0"], ["0.1", "0.1", "SparseDense", "1"], ["0.3", "0.3", "SparseDense", "1"], ["0.5", "0.5", "SparseDense", "1"], ["0.7", "0.7", "SparseDense", "1"], ["0.9", "0.9", "SparseDense", "1"], ["0.1", "0.1", "DD", "0"], ["0.3", "0.3", "DD", "0"], ["0.5", "0.5", "DD", "0"], ["0.7", "0.7", "DD", "0"], ["0.9", "0.9", "DD", "0"]]

repeats = 10
binary = "../../build/benchmark"
graph_name = "bert"

In [14]:
def load(file_name: str):
    results = []
    with open(file_name, "rt") as file:
        for line in file.readlines()[1:]:
            data = line.replace(' ', '').split(',')
            left_sparsity = float(data[1])
            right_sparsity = float(data[2])
            fmt = data[3]
            opt = data[4]
            analysis = float(data[7])
            load = float(data[8])
            comp = float(data[9])
            runtime = float(data[10])
            memory = float(data[11].replace('\n', ''))
            mean_metrics = {
                "config": f"{left_sparsity},{right_sparsity},{fmt},opt={opt}",
                "analysis": analysis, "load": load, "compilation": comp, "runtime": runtime, "memory": memory
            }
            results.append(mean_metrics)
    return results

In [15]:
def plotly(data):
    legend_map = {
        "SparseDense,opt=1": "sparse+prop",
        "SparseDense,opt=0": "sparse",
        "DD,opt=0": "dense",
    }

    configs = defaultdict(lambda: {"ratios": [], "runtime": [], "memory": [], "analysis": [], "compilation": []})
    ratio_labels = []

    for entry in data:
        parts = entry["config"].split(",")
        ratio = f"({parts[0]},{parts[1]})"
        config_type = ",".join(parts[2:])
        label = legend_map.get(config_type, config_type)

        if ratio not in ratio_labels:
            ratio_labels.append(ratio)

        configs[label]["ratios"].append(ratio)
        configs[label]["runtime"].append(entry["runtime"])
        configs[label]["memory"].append(entry["memory"])
        configs[label]["analysis"].append(entry["analysis"])
        configs[label]["compilation"].append(entry["compilation"])

    def ratio_key(r):
        vals = r.strip("()").split(",")
        return float(vals[0]), float(vals[1])
    ratio_labels = sorted(ratio_labels, key=ratio_key)

    # Create subplots
    fig = make_subplots(rows=3, cols=1,
                        shared_xaxes=False,
                        vertical_spacing=0.15,
                        subplot_titles=("Runtime vs Sparsity Ratio",
                                        "Memory Usage vs Sparsity Ratio",
                                        "Analysis vs Compilation vs Runtime (sparsity+prop)"))
    fig.layout.template = "plotly_white"
    # === 1) Runtime Plot (row 1) ===
    for label, values in configs.items():
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]
        fig.add_trace(
            go.Scatter(x=ratio_labels, y=runtime_values, mode="lines", name=f"Runtime ({label})", legendgroup="runtime"),
            row=1, col=1
        )

    # === 2) Memory Plot (row 2) ===
    for label, values in configs.items():
        memory_values = [v for _, v in sorted(zip(values["ratios"], values["memory"]), key=lambda x: ratio_key(x[0]))]
        fig.add_trace(
            go.Scatter(x=ratio_labels, y=memory_values, mode="lines", name=f"Memory ({label})", legendgroup="memory"),
            row=2, col=1
        )

    # === 3) Analysis vs Compilation vs Runtime (Bars) (row 3) ===
    for label, values in configs.items():
        if label != "sparse+prop":
            continue
        analysis_values = [v for _, v in sorted(zip(values["ratios"], values["analysis"]), key=lambda x: ratio_key(x[0]))]
        comp_values = [v for _, v in sorted(zip(values["ratios"], values["compilation"]), key=lambda x: ratio_key(x[0]))]
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]

        fig.update_yaxes(type="log", row=3, col=1)

        fig.add_trace(
            go.Bar(name=f"Analysis ({label})", x=ratio_labels, y=analysis_values, legendgroup="bar"),
            row=3, col=1
        )
        fig.add_trace(
            go.Bar(name=f"Compilation ({label})", x=ratio_labels, y=comp_values, legendgroup="bar"),
            row=3, col=1
        )
        fig.add_trace(
            go.Bar(name=f"Runtime ({label})", x=ratio_labels, y=runtime_values, legendgroup="bar"),
            row=3, col=1
        )

    # Layout adjustments
    fig.update_layout(
        height=1000,
        title_text="Benchmark Results",
        barmode='group',
        legend_tracegroupgap=300
    )
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=1, col=1)
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=2, col=1)
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=3, col=1)
    fig.update_yaxes(title_text="Runtime (s)", row=1, col=1)
    fig.update_yaxes(title_text="Memory (MB)", row=2, col=1)
    fig.update_yaxes(title_text="Log-scale Time (s)", row=3, col=1)

    fig.show()


In [16]:
def plotly_sparsity(data):
    configs = defaultdict(lambda: {"ratios": [], "runtime": []})
    ratio_labels = set()

    for entry in data:
        parts = entry["config"].split(",")
        config_type = ",".join(parts[2:])
        
        if config_type != "SparseDense,opt=1":
            continue

        if parts[0] != "0.0" and parts[1] != "0.0":
            label = "row/col"
            ratio = parts[0]
        elif parts[0] != "0.0":
            label = "row"
            ratio = parts[0]
        else:
            label = "col"
            ratio = parts[1]

        ratio_labels.add(ratio)
        configs[label]["ratios"].append(ratio)
        configs[label]["runtime"].append(entry["runtime"])

    ratio_labels = sorted(ratio_labels, key=lambda r: float(r))

    fig = go.Figure()
    for label, values in configs.items():
        sorted_data = sorted(zip(values["ratios"], values["runtime"]), key=lambda x: float(x[0]))
        sorted_ratios, sorted_runtimes = zip(*sorted_data)

        fig.add_trace(go.Scatter(
            x=sorted_ratios,
            y=sorted_runtimes,
            mode="lines",
            name=label
        ))

    fig.update_layout(
        title="Runtime vs Sparsity Ratio (Sparse+Prop)",
        xaxis_title="Sparsity Ratio",
        yaxis_title="Runtime (s)",
        template="plotly_white",
        height=500,
        legend=dict(
            title="Sparsity Type",
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        )
    )

    fig.show()


In [17]:
#data = run()
#data = random_run()
data = load("row_col_no_input.out")
#data = load("col_sparsity_batch_warmup.txt")
#data = load("row_col_sparsity_batch_warmup.txt")
#data.extend(load("col_sparsity_batch_warmup.txt"))
#data.extend(load("row_col_sparsity_batch_warmup.txt"))

In [18]:
plotly(data)

In [67]:
plotly_sparsity(data)